In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score, roc_curve, auc

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bts-v2(pakai).xlsx to bts-v2(pakai).xlsx


In [ ]:
df = pd.read_excel("bts-v2(pakai).xlsx", index_col = None)
df

,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari k...,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
df["sentimen"].value_counts()

1    501
0     29
Name: sentimen, dtype: int64

In [ ]:
#menghilangkan url
df["tweet"] = df["tweet"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari kaws,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
#remove mention
import re
def remove_mention(text):
    for i in text:
        result = re.sub("@[A-Za-z0-9_]+","",text)
    return result

df["tweet"] = df["tweet"].apply(remove_mention)
df

,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari kaws,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
#remove hashtags
def remove_hash(text):
    for i in text:
        result = re.sub("#[A-Za-z0-9_]+","",text)
    return result

df["tweet"] = df["tweet"].apply(remove_hash)
df

,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari kaws,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
#menghilangkan punctuation
df["tweet"] = df["tweet"].str.replace(r'[^\w\s]+', '')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari kaws,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
#remove emoji
df["tweet"] = df["tweet"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df

,tweet,sentimen
0,Jimin adalah artis pertama yang mencapai punca...,1
1,ForYouth merupakan lagu baru untuk penggemar B...,1
2,jhope terlihat menyukai kiriman terbaru dari kaws,1
3,album kedua disusun untuk menjelaskan BTS yang...,1
4,Daftar lagu album kedua untuk album antologi P...,1
...,...,...
525,Semua tiket PTD di panggung LasVegas 4 hari ud...,0
526,Ada beberapa army yang tudung nya udah sampai ...,0
527,Masih tidak bisa berkata kata pantas jimin dar...,0
528,kebayang ga didepan begitu banyak wartawan dan...,0


In [ ]:
#case folding
data_lower = []
for line in df["tweet"]:
  a = line.lower()
  data_lower.append(a) 

data_lower

['jimin adalah artis pertama yang mencapai puncak nomor satu di chart papan pengumuman minggu an baik sebagai penyanyi solo maupun dengan grup',
 'foryouth merupakan lagu baru untuk penggemar bts yaitu army lagu itu yang menunjukkan rasa kasih sayang dan terima kasih bts kepada army',
 'jhope terlihat menyukai kiriman terbaru dari kaws',
 'album kedua disusun untuk menjelaskan bts yang tetap bersinar setelah berjalan bersama selama sembilan tahun saling menghargai selera dan warna satu sama lain secara khusus lagu di album kedua dipilih oleh ketujuh anggota sendiri',
 'daftar lagu album kedua untuk album antologi proof berisi lagu solo dan lagu unit yang menunjukkan perbedaan warna pesona dan kisah tulus ketujuh anggota',
 'ketika bts memenangkan penghargaan diketahui bahwa mereka akan menyampaikan kesan mereka melalui video',
 'lagu tersebut diperbarui kembali untuk dimasukkan dalam album bts yaitu proof',
 'taehyung berkomentar dikiriman jhope taehyung bertanya dimana tempat yang ada

In [ ]:
#remove number
import re
data_no_number = []
for line in data_lower:
  result = re.sub(r'\d', '', line)
  data_no_number.append(result)

data_no_number

['jimin adalah artis pertama yang mencapai puncak nomor satu di chart papan pengumuman minggu an baik sebagai penyanyi solo maupun dengan grup',
 'foryouth merupakan lagu baru untuk penggemar bts yaitu army lagu itu yang menunjukkan rasa kasih sayang dan terima kasih bts kepada army',
 'jhope terlihat menyukai kiriman terbaru dari kaws',
 'album kedua disusun untuk menjelaskan bts yang tetap bersinar setelah berjalan bersama selama sembilan tahun saling menghargai selera dan warna satu sama lain secara khusus lagu di album kedua dipilih oleh ketujuh anggota sendiri',
 'daftar lagu album kedua untuk album antologi proof berisi lagu solo dan lagu unit yang menunjukkan perbedaan warna pesona dan kisah tulus ketujuh anggota',
 'ketika bts memenangkan penghargaan diketahui bahwa mereka akan menyampaikan kesan mereka melalui video',
 'lagu tersebut diperbarui kembali untuk dimasukkan dalam album bts yaitu proof',
 'taehyung berkomentar dikiriman jhope taehyung bertanya dimana tempat yang ada

In [ ]:
#remove whitespace
data_no_whitespace = []
for line in data_no_number:
  if len(line.split()) > 1:
    result = ' '.join(line.split())
    data_no_whitespace.append(result)

data_no_whitespace

['jimin adalah artis pertama yang mencapai puncak nomor satu di chart papan pengumuman minggu an baik sebagai penyanyi solo maupun dengan grup',
 'foryouth merupakan lagu baru untuk penggemar bts yaitu army lagu itu yang menunjukkan rasa kasih sayang dan terima kasih bts kepada army',
 'jhope terlihat menyukai kiriman terbaru dari kaws',
 'album kedua disusun untuk menjelaskan bts yang tetap bersinar setelah berjalan bersama selama sembilan tahun saling menghargai selera dan warna satu sama lain secara khusus lagu di album kedua dipilih oleh ketujuh anggota sendiri',
 'daftar lagu album kedua untuk album antologi proof berisi lagu solo dan lagu unit yang menunjukkan perbedaan warna pesona dan kisah tulus ketujuh anggota',
 'ketika bts memenangkan penghargaan diketahui bahwa mereka akan menyampaikan kesan mereka melalui video',
 'lagu tersebut diperbarui kembali untuk dimasukkan dalam album bts yaitu proof',
 'taehyung berkomentar dikiriman jhope taehyung bertanya dimana tempat yang ada

In [ ]:
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 3.9 MB/s 


In [ ]:
#stemming untuk teks berbahasa Indonesia
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

translator = str.maketrans("", "", string.punctuation)
data_stemmed = map(lambda x: stemmer.stem(x), data_no_whitespace)
data_no_punc = map(lambda x: x.lower().translate(translator), data_stemmed)

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#stopwords removal
stopword = stopwords.words("indonesian")
stopword.extend(["bts", "ya", "allah", "omg", "oh", "wkwk", "hahaha", "hehehe", "guys", "op", "dm", "x", 
                 "nya", "bro", "nah", "eh", "ga", "wkwkwk", "tak", "hei", "kah","nih", "ku", "yah", "tuh", 
                 "aduh", "yes", "huhu", "meow", "ohh", "ngeh", "kurleb", "ni", "otw", "ah", "woahhh", "yuk", 
                 "kanjeng", "loh", "lol", "entahlah", "kekekeke", "nge", "pun", "crsoorooroop", "udah", 
                 "oppa", "om", "mas", "hmm", "yah", "yg", "heh", "guys", "dong", "aye", "waw", "q", "kan", 
                 "bro", "maknae", "yep", "aegyo", "uncut", "mu", "nge", "pun", "n", "tgl", "wib", "abar", 
                 "abis", "yukk", "k", "kst", "thn", "pcs", "jt", "m", "n", "an", "nya", "di", "in", 
                 "bang", "dear", "gaada", "dgn", "yaampun", "sih", "gue", "kek", "i", "pun", 
                 "mas", "mbak", "lah", "gatau", "u", "nge", "bgt", "aka", "kak", "tak", "btw", "nih", "daddy", 
                 "my", "god", "u", "yes", "wow", "congratz", "hai", "okay", "waw", "th", "dm", "st", "rd", "pm", 
                 "able", "zombie", "your", "you", "abang", "yey", "yourself", "yet", "bilang", "lihat", "tolong", "gitu", "situ", "lari", "tangan", "pas", "cari", "biar", "oke", "on",
                 "pakai", "ayah", "pas", "juta", "jam", "menit", "orang", "ubah", "papa", "by", "lupa", "mari", "duluan", "betapa", "ubah", "to", "anak", "pilih", "selamat", "ulang", "tahun",
                 "bikin", "gimana", "gin", "amp", "aja", "yaa", "tau", "hehe", "halo", "iya", "ngga", "so", "of", "kalo", "moga", "lo", "gak", "banget", "pokok", "emang", "maaf", "aju", "gas", 
                 "sanggat", "nang", "la", "hello", "us", "deh", "the", "emang", "am", "pm"])

datafinal = []
for line in data_no_punc:
  word_token = nltk.word_tokenize(line)
  word_token = [word for word in word_token if word not in stopword and not word[0].isdigit()]
  datafinal.append(" ".join(word_token))

datafinal

['jimin artis capai puncak nomor chart papan minggu nyanyi solo grup',
 'foryouth rupa lagu gemar army lagu kasih sayang terima kasih army',
 'jhope suka kirim kaws',
 'album susun sinar jalan sembilan harga selera warna khusus lagu album tujuh anggota',
 'daftar lagu album album antologi proof isi lagu solo lagu unit beda warna pesona kisah tulus tujuh anggota',
 'menang harga kesan video',
 'lagu masuk album proof',
 'taehyung komentar kirim jhope taehyung kirim jhope',
 'rm mono lampau streaming spotify',
 'jhope instastory cinta jhope album tanda nyanyi crush',
 'cuplik pikir momen indah kim taehyung break silence movie commentary album proof',
 'kartu foto tema sampul album album proof',
 'dukung nyanyi album harga henti protes nuntut',
 'taehyung suka panggil video suka kirim pesan telpon jimin suka kirim pesan suka telpon jungkook suka kirim pesan telpon jin suka kirim pesan simpul ngapain handphone',
 'terang tulis suga kirim weverse rupa buah lirik lagu kenal korea',
 'suga ki

In [ ]:
#count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_train = CountVectorizer()
vectorize_train = vectorizer_train.fit_transform(datafinal)
vectorize_train = vectorize_train.toarray()
feature_vectorize_train = vectorizer_train.get_feature_names()
count_vector_train = pd.DataFrame(vectorize_train, columns=feature_vectorize_train)
count_vector_train.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abyss,acara,adegan,adu,agens,agustd,ahli,air,ajaib,ajak,...,withyou,world,wujud,yard,yeonjun,yoongi,yoongijung,youtube,yumnyam,zaman
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer(use_idf=True)
tfidf_ = tfidf_transform.fit_transform(vectorize_train)
tfidftrain = pd.DataFrame(tfidf_.A, columns=vectorizer_train.get_feature_names())
tfidftrain

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abyss,acara,adegan,adu,agens,agustd,ahli,air,ajaib,ajak,...,withyou,world,wujud,yard,yeonjun,yoongi,yoongijung,youtube,yumnyam,zaman
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x = tfidf_
y = df["sentimen"]

In [ ]:
#stratified k-fold cv
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
i = 1
for train_index, test_index in skf.split(x,y):
  print("Fold : ", i)
  print("TRAIN:", train_index, "\nTEST:", test_index)
  x_train, x_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]
  i += 1
  model = BernoulliNB()
  model.fit(x_train, y_train)
  # print("---------------------")
  # y_pred_prob_train = model.predict_proba(x_train)
  # print("NB classification probability for training data : \n\n", y_pred_prob_train)
  print("---------------------")
  y_pred_train = model.predict(x_train)
  cm_train = confusion_matrix(y_train, y_pred_train)
  print("Train Confusion matrix\n\n", cm_train)
  print("---------------------")
  TP_train = cm_train[0,0]
  TN_train = cm_train[1,1]
  FP_train = cm_train[0,1]
  FN_train = cm_train[1,0]
  accuracy_train = (TP_train+TN_train)/float(TP_train+TN_train+FP_train+FN_train)
  print("Train Accuracy : {0:0.4f}".format(accuracy_train))
  print("---------------------")
  AUC_train = roc_auc_score(y_train, y_pred_train)
  print("Train AUC : {:.4f}".format(AUC_train))
  print("---------------------")
  y_pred_prob_test = model.predict_proba(x_test)
  print("NB classification probability for testing data : \n\n", y_pred_prob_test)
  print("---------------------")
  y_pred_test = model.predict(x_test)
  cm_test = confusion_matrix(y_test, y_pred_test)
  print("---------------------")
  print("Test Confusion matrix\n\n", cm_test)
  print("---------------------")
  TP_test = cm_test[0,0]
  TN_test = cm_test[1,1]
  FP_test = cm_test[0,1]
  FN_test = cm_test[1,0]
  accuracy_test = (TP_test+TN_test)/float(TP_test+TN_test+FP_test+FN_test)
  print("Test Accuracy : {0:0.4f}".format(accuracy_test))
  print("---------------------")
  AUC_test = roc_auc_score(y_test, y_pred_test)
  print("Test AUC : {:.4f}".format(AUC_test))

Fold :  1
TRAIN: [  0   1   2   3   4   5   6   7   8   9  11  12  13  14  15  16  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  35  36  37  38
  39  40  41  42  43  45  48  50  51  52  53  54  55  56  58  59  60  61
  62  63  64  66  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98 100
 101 103 104 105 106 107 108 109 110 111 112 114 115 116 117 118 120 121
 122 123 124 125 126 127 128 129 131 132 133 134 135 136 139 140 141 142
 143 144 145 147 148 149 151 154 155 157 158 159 160 161 162 164 165 168
 169 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 223 224
 225 226 227 228 229 230 231 232 234 235 236 237 238 239 240 241 242 244
 245 246 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263
 264 265 266 267 268 269 270 271 2

In [ ]:
#smote
sm = SMOTE()
x_res, y_res = sm.fit_resample(x, y)
x_train_res, x_test_res, y_train_res, y_test_res = train_test_split(x_res, y_res, test_size=0.25, random_state=42)
model = BernoulliNB()
model.fit(x_train_res, y_train_res)
y_pred_res_train = model.predict(x_train_res)
print("Train Accuracy: {:.2f}%".format(accuracy_score(y_train_res, y_pred_res_train) * 100))
print("Train AUC Score: {:.2f}%".format(roc_auc_score(y_train_res, y_pred_res_train) * 100))
print("\nTrain Confusion Matrix:\n", confusion_matrix(y_train_res, y_pred_res_train))
print("---------------------")
y_pred_prob_test = model.predict_proba(x_test_res)
y_pred_res_test = model.predict(x_test_res)
print("NB classification probability for testing data : \n\n", y_pred_prob_test)
print("Testing Accuracy: {:.2f}%".format(accuracy_score(y_test_res, y_pred_res_test) * 100))
print("Testing AUC Score: {:.2f}%".format(roc_auc_score(y_test_res, y_pred_res_test) * 100))
print("\nTesting Confusion Matrix:\n", confusion_matrix(y_test_res, y_pred_res_test))

Train Accuracy: 99.73%
Train AUC Score: 99.73%

Train Confusion Matrix:
 [[375   0]
 [  2 374]]
---------------------
NB classification probability for testing data : 

 [[3.99048558e-09 9.99999996e-01]
 [1.00000000e+00 3.92774405e-13]
 [1.00000000e+00 3.15632197e-15]
 [1.00000000e+00 4.17031314e-13]
 [9.99996502e-01 3.49765833e-06]
 [1.00000000e+00 7.45139641e-13]
 [9.99999992e-01 7.95797195e-09]
 [2.47531723e-04 9.99752468e-01]
 [9.99999995e-01 4.78845571e-09]
 [1.36640077e-03 9.98633599e-01]
 [1.00000000e+00 3.96088277e-14]
 [1.92361268e-03 9.98076387e-01]
 [1.00000000e+00 4.99192980e-13]
 [1.00000000e+00 6.99180874e-14]
 [1.00000000e+00 1.12255301e-11]
 [1.00000000e+00 5.18532563e-13]
 [1.15290807e-02 9.88470919e-01]
 [1.00000000e+00 5.18532563e-13]
 [1.00000000e+00 2.16639061e-15]
 [2.80890463e-05 9.99971911e-01]
 [9.99999883e-01 1.16875392e-07]
 [1.16251208e-06 9.99998837e-01]
 [9.55494637e-05 9.99904451e-01]
 [9.99938381e-01 6.16185966e-05]
 [3.48924410e-04 9.99651076e-01]
 [1.5